In [1]:
using BasisFunctions
using FrameFun
using DomainSets
using Plots
using LinearAlgebra
gr()

INFO: Recompiling stale cache file /home/daan/.julia/lib/v0.6/BasisFunctions.ji for module BasisFunctions.
ERROR: LoadError: ArgumentError: Module DomainSets not found in current path.
Run `Pkg.add("DomainSets")` to install the DomainSets package.
Stacktrace:
 [1] _require(::Symbol) at ./loading.jl:435
 [2] require(::Symbol) at ./loading.jl:405
 [3] include_from_node1(::String) at ./loading.jl:576
 [4] include(::String) at ./sysimg.jl:14
 [5] anonymous at ./<missing>:2
while loading /home/daan/.julia/v0.6/BasisFunctions/src/BasisFunctions.jl, in expression starting on line 3


LoadError: [91mFailed to precompile BasisFunctions to /home/daan/.julia/lib/v0.6/BasisFunctions.ji.[39m

## Use AZ algorithm to solve using platform.

Platform implements matrix_A(...) and matrix_Zt(...) routines

In [2]:
function myFun(f::Function,P::Platform,i)
    AZS = AZSolver(matrix_A(P,i),matrix_Zt(P,i))
    F = DictFun(primal(P,i),AZS*f)
end

myFun (generic function with 1 method)

### Fourier Basis platform

In [3]:
P = BA.fourier_platform()


BasisFunctions.GenericPlatform(nothing, BasisFunctions.Fourier{Float64}, BasisFunctions.Fourier{Float64}, BasisFunctions.#513, BasisFunctions.#514, BasisFunctions.OddDoublingSequence(1), "Fourier series")

works for periodic functions

In [ ]:
f = x->cos(4*pi*x)
F = myFun(f,P,8)
plot(F)

but not for general functions

In [ ]:
f = x->exp(x)
F = myFun(f,P,8)
plot(F)

Singular value profile

In [ ]:
plot(svdvals(matrix(matrix_A(P,5)*matrix_Zt(P,5))),ylims=(-0.1,1.1))

### Extension platform - from oversampled platform

In [ ]:
D = 0..0.5
P2a = BasisFunctions.fourier_platform(oversampling=4)
P2 = FrameFun.extension_frame_platform(P2a,D)

In [ ]:
f = x->exp(x)
F2 = myFun(f,P2,8)
plot(F2)

Singular value profile

In [ ]:
plot(svdvals(matrix(matrix_A(P2,5)*matrix_Zt(P2,5))),ylims=(-0.1,1.1))

## Weighted Sum Platform based on extension platform

Square root type singularity

In [ ]:
WSP = FrameFun.WeightedSumPlatform([x->sqrt(x),x->1],P2)

In [ ]:
f = x->sqrt(x)*(1-x)-exp(x)
FSP = myFun(f,WSP,8)
plot(FSP,f)

This is much more accurate than the regular extension platform

In [ ]:
FSPb = myFun(f,P2,8)
plot(FSP,f)
plot!(FSPb,f)

Singular value profile

In [ ]:
plot(svdvals(matrix(matrix_A(WSP,7)*matrix_Zt(WSP,7))),ylims=(-0.1,4.1))